In [1]:
import requests
import time
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import json
import sqlite3
from sqlite3 import Error
from queue import Queue 

In [2]:
# terms_dict = defaultdict(list)
terms_list = []
appDictionary = {}
relatedSearchTerms = {}

In [3]:
def commaSeparated(term):
    result = "";
    if len(term.split(' ')) > 1:
        count = len(term.split(' '));
        i = 0;
        for word in term.split(' '):
            if i > 0:
                result = result + "+";
            result = result + word;
            i = i + 1;
        return result;
    else:
        return term;

In [4]:
#Class to store app details in object format
class AppDetails:
    def __init__(self, title, description, stars, appID, imageSource, developerName):
        self.title = title
        self.description = description
        self.stars = stars
        self.appID = appID
        self.imageSource = imageSource
        self.developerName = developerName
        
    def toString(application):
        return application.title + ", " + application.appID + ", " + application.description + ", " + application.stars + ", " + application.imageSource + ", " + application.developerName

In [5]:
#DB DRIVERS

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
            
            
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
sql_create_appDetailsMainTable_table = """ CREATE TABLE IF NOT EXISTS appDetailsMainTable (
                                        key text NOT NULL,
                                        appIDs text NOT NULL,
                                        relatedSearchTerms text,
                                        source text NOT NULL
                                    ); """

sql_create_appDetails_table = """ CREATE TABLE IF NOT EXISTS appDetails (
                                        appID text NOT NULL,
                                        title text NOT NULL,
                                        description text,
                                        stars text,
                                        imageSource text,
                                        developerName text,
                                        source text
                                    ); """

sql_create_relatedSearchTerms_table = """ CREATE TABLE IF NOT EXISTS relatedSearchTerms (
                                        keyTerm text PRIMARY KEY,
                                        terms text NOT NULL,
                                        source text
                                    ); """

def databaseStartUp():
    # Create connection
    conn = create_connection(r"apkplz.db")

    # Create table
    if conn is not None:
        create_table(conn, sql_create_appDetailsMainTable_table)
        create_table(conn, sql_create_appDetails_table)
        create_table(conn, sql_create_relatedSearchTerms_table)
        print("Database created!")
    else:
        print("Error! cannot create the database connection.")
    
    return conn;
        
        
def insertIntoAppDetailsMainTable(conn, task):
 
    sql = ''' INSERT INTO appDetailsMainTable(key,appIDs,relatedSearchTerms,source)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

def insertIntoAppDetails(conn, task):
 
    sql = ''' INSERT INTO appDetails(appID, title, description, stars, imageSource, developerName, source)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid
     
conn = databaseStartUp()

Database created!


In [6]:
with open('android_terms.csv','rt')as f:
  data = csv.reader(f)
  lineNumber = 0;
  for row in data:
        if lineNumber >= 1:
            key = row[2];
            terms = row[3];
            i = 0;
            for term in terms.split('"'):
                if i%2 == 0:
                    i = i+1;
                    continue
                else:
                    i = i+1;
                    terms_list.append(term)
        else:
            lineNumber = lineNumber + 1;

In [7]:
finalTermsList = []

In [8]:
finalTermsList.clear()
for term in terms_list:
    result = commaSeparated(term)
    finalTermsList.append(result)

In [9]:
len(finalTermsList)

432331

In [ ]:
# WRITTEN FOR TESTING PURPOSE PLEASE IGNORE 

payload = {'q': 'mspy', 't': 'app'}
r = requests.get('https://apk.support/search', params=payload);
print(r.url)
print(r.status_code)

In [ ]:
# TEST
soup = BeautifulSoup(r.text, 'html.parser')
app_table = soup.find_all("h3")
appNamesList = []
for app in app_table:
    appName = app.get_text()
    appNamesList.append(appName)
appDictionary["mspy"] = appNamesList


# TEST Cleanup List
for term in finalTermsList:
    if(term == ""):
        finalTermsList.remove(term)
singleWordSet = set();
for term in finalTermsList:
    if(term.find("+") == -1 and term != ""):
        singleWordSet.add(term)

In [10]:
q = Queue();
wordSet = set();
for term in finalTermsList:
    if(term != ""):
        wordSet.add(term)
print(len(wordSet))
for word in wordSet:
    q.put(word)
print(q.qsize())

1557
1557


In [ ]:
# FOR TEST PURPOSE
singleWordSet = {'cheater', 'cheaters', 'cheating', 'track husband', 'keylogger', 'keyloggers'}

In [ ]:
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word, 't': 'app'};
    r = requests.get('https://apk.support/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "it_column"})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    for name in names_table:
        # Developer Information
        developerPart = name.find_all("div", attrs={"class": "ss_tg"})
        developerPart = developerPart[0].find_all("a")
        developerTag = developerPart[0]['href']
        developerTag = developerTag[10:]
        developerName = developerPart[0].get_text()
        information = name.find_all("a")
        # Title
        titleTag = information[0].find_all("h3")
        title = titleTag[0].get_text()
#         print("Title is " + title)
        # Description
        descriptionTag = information[0].find_all("p")
        description = descriptionTag[0].get_text()
#         print("Description is " + description)
        # Stars
        starsTag = information[0].find_all("div", attrs = {"class" : "stars"})
        starsSpan = starsTag[0].find_all("span")
        stars = starsSpan[0]['title']
        starCount = stars[stars.rindex(' ')+1:]
#         print("Stars is " + starCount)
        # AppID
        appID = information[0]['href']
        appID = appID[4 : ]
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
#         print("AppID:- " + appID)
        # Image Source Link
        imageTag = information[0].find_all("div", attrs={"class" : "seo_img"})
        imageTag = imageTag[0].find_all("img")
        imageSource = imageTag[0]['data-original']
#         print(" ImageSrc:- " + imageSource)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, stars, imageSource, developerName, 'apk.support');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        
    # Suggestion Addition
    suggestionList = soup.find_all("div", attrs={"class": "suggest"})
    suggestionList = suggestionList[0].find_all("li")
    suggestions = []
    suggestionsString = "";
    i = 0
    for suggestion in suggestionList:
        suggestionName = suggestion.get_text()
        if (i != 0):
            suggestionsString = suggestionsString + ","
        suggestionsString = suggestionsString + suggestionName
        i = 1
        suggestions.append(suggestionName)
        modifiedSuggestionName = commaSeparated(suggestionName)
        if(modifiedSuggestionName not in wordSet):
#             print("Added " + modifiedSuggestionName)
            wordSet.add(modifiedSuggestionName)
            q.put(modifiedSuggestionName)
#         else:
#             print("Not Adding " + modifiedSuggestionName)
    
    #Insert Into Main Table
    taskMainTable = (word, appIDList, suggestionsString, 'apk.support');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
    
    
#     appDictionary[word] = finalList
#     relatedSearchTerms[word] = suggestions
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    print("Completed " + word)

In [ ]:
#https://apkgk.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'keyword': word}
    r = requests.get('https://apkgk.com/search/', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appId = soup.find_all("ul", attrs={"class":'topic-wrap'})
    if(len(appId) == 0):
        print("Skipping " + word)
        continue
    appId = appId[0].find_all("a")
    names_table = soup.find_all('div', attrs={"class":'topic-bg'})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    counter = 0
    for name in names_table:
        appName = name.find_all("div", attrs={"class": "topic-tip-name"})
        appDesc = name.find_all("div", attrs={"class": "topic-tip-description"})
        appSrcMain  = name.find_all("div", attrs={"class": "c-lz-load"})
        imageTag = appSrcMain[0].find_all("img")
        appLink = imageTag[0]['data-src']
        #for img in appSrcMain:
        print(appId[counter]['href'])
        print(appLink)
        print(appName[0].get_text())
        print(appDesc[0].get_text())
        
        appID = appId[counter]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        imageSource = appLink
        description = appDesc[0].get_text()
        title = appName[0].get_text()
        starCount = "NULL"
        developerName = "NULL"
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkgk.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkgk.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
        


In [ ]:
#https://www.malavida.com
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    word = word.replace('+','-')
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    #payload = {'/':word}
    r = requests.get('https://www.malavida.com/en/s/'+word);
    soup = BeautifulSoup(r.text, 'html.parser');
    appDetails = soup.find_all("section", attrs={"class":'app-list'})
    appList = soup.find_all("section", attrs={"class":'app-download'})
    counter = 0
    finalList = []
    appIDList = ""
    first = 0
    notFound = soup.find_all("section", attrs={"class":'not-found'})
    if(len(notFound)>0):
        print("Skipping " + word)
        continue
    print(len(appList))
    for app in appList:
        appSrc = app.find_all("div", attrs={"class":"title"})
        appDesc = app.find_all("p")
        imageSource = app.find_all('img')
        imageSource = imageSource[0]['src']
        appLink = appSrc[0].find_all("a")
        appID = appLink[0]['href']
        title = appLink[0].get_text()
        description = appDesc[0].get_text()
        counter = counter+1
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        starCount = "NULL"
        developerName = "NULL"
        print(title +", "+description+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'malavida.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'malavida.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
    
    

In [ ]:
#https://apkfab.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkfab.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appList = soup.find_all("div", attrs={"class":'list'})
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        title = app.find_all("div",attrs={"class":"title"})
        if(len(title)  < 1):
            continue
        title = title[0].get_text()
        rating = app.find_all("span", attrs={"class":"rating"})
        starCount = rating[0].get_text()
        description = "NULL"
        developerName = "NULL"
        imageSource = app.find_all("img")
        imageSource  = imageSource[0]['data-src']
        appID = app.find_all("a")
        appID = appID[0]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkfab.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        #counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkfab.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        


In [ ]:
#https://apkdom.com/
#Not working
qTrial = Queue()
qTrial.put("cheating")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(1)
    word = word.replace('+','%20')
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'': word};
    print(payload)
    r = requests.get('https://apkdom.com/Search/', params=payload);
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser');
    print(soup)
    appList = soup.find_all("section", attrs={"class":"LeftSection"})
    
    #print(appList)
    appList = appList[0].find_all("div", attrs={"class":'NewBlock1'})
    #print(appList)
    #appList = appList[0].find_all("div", attrs={"class":'AppCont'})
    #print(appList)
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        title = app.find_all("h3",attrs={"class":"top14"})
        if(len(title)  < 1):
            continue
        starCount = "NULL"
        description = "NULL"
        developerName = "NULL"
        imageSource = app.find_all("img")
        imageSource  = imageSource[0]['data-src']
        appID = app.find_all("a")
        appID = appID[0]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        
    
    


In [ ]:
#https://m.allfreeapk.com/
qTrial = Queue()
qTrial.put("cheating")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    print(payload)
    r = requests.get('https://m.allfreeapk.com/search.html?', params=payload);
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser');
    appList = soup.find_all("div", attrs={"class":"list"})
    appList = appList[0].find_all("li");
    #print(appList)
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        appDetails  = app.find_all("div",attrs={"class":"l"}) 
        title =  app.find_all("div",attrs={"class":"r"}) 
        title = title[0].find_all("a")
        title = title[0].get_text()
        #print(title)
        starCount = "NULL"
        description = "NULL"
        developerName = "NULL"
        imageSource = appDetails[0].find_all("img")
        imageSource = imageSource[0]["data-original"];
        appID = appDetails[0].find_all("a")
        appID = appID[0]["href"]
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'm.allfreeapk.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        #counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'm.allfreeapk.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
    

In [ ]:
#https://apktada.com/
qTrial = Queue()
qTrial.put("cheating")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    print(payload)
    r = requests.get('https://apktada.com/search?', params=payload);
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser');
    appList = soup.find_all("div", attrs={"class":"section row nop-sm"})
    appList = appList[0].find_all("div",attrs={"class":"row itemapp"});
    #print(appList)
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        appDetails  = app.find_all("div",attrs={"class" : "col-md-12 col-sm-9 vcenter apptitle"}) 
        title = appDetails[0].find_all("a")
        title = title[0]['title']
        #print(title)
        starCount = "NULL"
        description = "NULL"
        developerName = "NULL"
        imageSource = app.find_all("div",attrs={"class" : "col-md-12 col-sm-3 vcenter"}) 
        imageSource = imageSource[0].find_all("img");
        imageSource = imageSource[0]["data-original"];
        #print(imageSource)
        appID = app.find_all("div",attrs={"class" : "col-md-12 col-sm-3 vcenter"}) 
        appID = appID[0].find_all("a")
        appID = appID[0]["href"]
        #print(appID)
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apktada.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        #counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apktada.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
        

In [15]:
#https://apkplz.net/
qTrial = Queue()
qTrial.put("cheating")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkplz.net/search?', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appList = soup.find_all("div", attrs={"class":"section row nop-sm"})
    appList = appList[0].find_all("div",attrs={"class":"row itemapp"});
    #print(appList)
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        appDetails  = app.find_all("div",attrs={"class" : "col-md-12 col-sm-9 vcenter apptitle"}) 
        title = appDetails[0].find_all("a")
        title = title[0]['title']
        #print(title)
        starCount = "NULL"
        description = "NULL"
        developerName = "NULL"
        imageSource = app.find_all("div",attrs={"class" : "col-md-12 col-sm-3 vcenter"}) 
        imageSource = imageSource[0].find_all("img");
        imageSource = imageSource[0]["data-original"];
        #print(imageSource)
        appID = app.find_all("div",attrs={"class" : "col-md-12 col-sm-3 vcenter"}) 
        appID = appID[0].find_all("a")
        appID = appID[0]["href"]
        #print(appID)
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkplz.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        #counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkplz.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
    

Starting call+record+app+downloading 0 with queue length 849
Starting free+calls+and+sms+track 0 with queue length 848
Starting tmobile+family+tracker 0 with queue length 847
Starting auto+call+recorder+2017+samsung 0 with queue length 846
Starting track+cheating+girlfriend 0 with queue length 845
Starting track+a+phone+by+the+number 0 with queue length 844
Starting text+message+sync 0 with queue length 843
Starting call+recorder+for+android+phone 0 with queue length 842
Starting gps+tracking+app 0 with queue length 841
Starting call+recorder+application 0 with queue length 840
Starting latest+call+recorder+for+android+phones 0 with queue length 839
Starting Auto+Call+Recorder+Pro+2018 0 with queue length 838
Starting spyrix+free+keylogger 0 with queue length 837
Starting find+my+friends+phone+location+by+phone+number 0 with queue length 836
Starting Automatic+Call+Recorder+&+Hide+App+Pro+-+callBOX 0 with queue length 835
Starting free+sms+tracker+app 0 with queue length 834
Starting c

Starting auto+call+voice+record+files+open+app+for+android+mob 0 with queue length 723
Starting control+my+kids+phone 0 with queue length 722
Starting track+my+lost+phone+by+phone+number 0 with queue length 721
Starting spy+on+my+girlfriend 0 with queue length 720
Starting locate+phone+find+my+lost+phone+by+phone+number 0 with queue length 719
Starting att+message+backup+and+sync 0 with queue length 718
Starting track+cell+phone+location+by+mobile+number+or+imei+number 0 with queue length 717
Starting auto+call+recorder+pro+free 0 with queue length 716
Starting track+boyfriend+text+messages 0 with queue length 715
Starting backuptrans+sms+sync 0 with queue length 714
Starting where+is+my+boyfriend+tracker+free 0 with queue length 713
Starting call+record+app+lock 0 with queue length 712
Starting track+other+cell+phone+numbers 0 with queue length 711
Starting track+location+by+imei+number 0 with queue length 710
Starting keylogger+spy 0 with queue length 709
Starting catch+my+spouse+che

Starting catch+cheating+spouse 0 with queue length 595
Starting one+touch+call+recorder+for+android+mobile 0 with queue length 594
Starting find+phone+location+free 0 with queue length 593
Starting how+to+track+my+boyfriend's+phone 0 with queue length 592
Starting track+husband's+phone 0 with queue length 591
Starting spy+on+my+boyfriend 0 with queue length 590
Starting find+my+phone+app+apple+iphone 0 with queue length 589
Starting control+lg+tv+with+phone 0 with queue length 588
Starting family+trackers+free 0 with queue length 587
Starting call+recorder+-+automaticcall+rec 0 with queue length 586
Starting free+call+and+sms+tracker+apps 0 with queue length 585
Starting free+app+facebook+for+friends+sync+to+phone+contact+list 0 with queue length 584
Starting Call+Recorder+Automatic++2018 0 with queue length 583
Starting track+mobile+number+location+apps 0 with queue length 582
Starting phone+sync+nextcloud 0 with queue length 581
Starting call+recorder+appliqato 0 with queue length 58

Starting call+recorder+-+automatic+free+app+2018 0 with queue length 468
Starting free+sms+and+call+tracker 0 with queue length 467
Starting my+children+bus+tracker 0 with queue length 466
Starting automatic+call+recorder+1000+pro+free 0 with queue length 465
Starting spy+on+my+boyfriends+phone 0 with queue length 464
Starting call+recorder+app+for+samsung+j7+pro 0 with queue length 463
Starting phone+number+track+app 0 with queue length 462
Starting head+phone+controler 0 with queue length 461
Starting spy+tracker+on+cheating+spouse 0 with queue length 460
Starting listen+to+911+calls 0 with queue length 459
Starting call+recorder+hide+app+for+android 0 with queue length 458
Starting boyfriend+tracker+by+number 0 with queue length 457
Starting a+free+hidden+app+that+can+locate+and+track+a+cell+phone+by+their+text+and+calls 0 with queue length 456
Starting spy+my+husband 0 with queue length 455
Starting automatic+call+record+apps+download 0 with queue length 454
Starting auto+sync+sms 

Starting automatic+call+recorder+for+samsung+galaxy+s7+edge 0 with queue length 343
Starting find+phone+location 0 with queue length 342
Starting call+and+sms+tracke 0 with queue length 341
Starting family+tracker+360+free 0 with queue length 340
Starting track+wife+phone 0 with queue length 339
Starting app+to+catch+cheaters 0 with queue length 338
Starting is+my+husband+cheating+app 0 with queue length 337
Starting free+phone+number+tracker+with+location 0 with queue length 336
Starting samsung+auto+call+recorder+2017 0 with queue length 335
Starting track+mobile+phone+exact+location+and+read+sms+and+view+call+log 0 with queue length 334
Starting track+my+boyfriend+phone 0 with queue length 333
Starting find+location+by+phone+number 0 with queue length 332
Starting call+recording+app+free 0 with queue length 331
Starting how+to+track+a+cell+phone+location+of+boyfriend+without+him+knowing 0 with queue length 330
Starting auto+call+recorder+app+for+samsung 0 with queue length 329
Start

Starting gps+find+my+child+tracker 0 with queue length 217
Starting sync+phone+to+smart+watch 0 with queue length 216
Starting find+my+lost+phone+by+phone+number 0 with queue length 215
Starting Call+Recorder+S9+-+Automatic+Call+Recorder+Pro 0 with queue length 214
Starting app+children+tracker 0 with queue length 213
Starting Whistle+Find+My+Phone+easy 0 with queue length 212
Starting children+tracker+beta 0 with queue length 211
Starting my+wife+is+cheating+on+me+app 0 with queue length 210
Starting track+location+of+incoming+calls 0 with queue length 209
Starting all+call+recorder+automatically 0 with queue length 208
Starting catch+a+cheating+lover 0 with queue length 207
Starting girlfriend+mobile+call+and+sms+track 0 with queue length 206
Starting hackers+keyloggers+wifi 0 with queue length 205
Starting track+a+number+by+phone 0 with queue length 204
Starting free+family+gps+tracker 0 with queue length 203
Starting record+my+call+app+download 0 with queue length 202
Starting trac

Starting latest+call+recorder+for+android+phones+2017 0 with queue length 92
Starting track+my+kids+phones+with+gps 0 with queue length 91
Starting best+free+find+my+phone+app 0 with queue length 90
Starting track+my+wife's+cell+phone 0 with queue length 89
Starting family+locator+-+gps+tracker+free 0 with queue length 88
Starting boyfriend+cheating+on+girlfriend+games 0 with queue length 87
Starting a+free+hidden+sms+mms+text+tracker 0 with queue length 86
Starting video+call+recorder+-+automatic+call+recorder+freevi 0 with queue length 85
Starting Backuptrans+SMS+Sync 0 with queue length 84
Starting boyfriend+cheating+app 0 with queue length 83
Starting cheaters+apps 0 with queue length 82
Starting track+cheating+spouse+cell+phone 0 with queue length 81
Starting track+my+spouse+phone 0 with queue length 80
Starting hackers+keyloggers 0 with queue length 79
Starting how+to+catch+a+cheating+spouse 0 with queue length 78
Starting call+recorder+free+apps 0 with queue length 77
Starting t

In [28]:
# Map check Code
for app in appDictionary:
    appList = appDictionary[app]
    print("Entries for " + app)
    for application in appList:
        print(application.toString())
        print("------------------------------->>>")

In [ ]:
# Saving dict to file
with open('apksupport.json', 'w') as f:
    json.dump(appDictionary, f)

with open('relatedSearchTerms_apksupport.json', 'w') as f:
    json.dump(relatedSearchTerms, f)

In [ ]:
SQLite3
dataset package python

upper table -> key and set of app ids
lower table -> app id is primary key and we write to this.

In [ ]:
conn.close()

In [ ]:
# FOR APKCOMBO.COM (ERROR: CLOUDFLARE BLOCKING)
qTrial = Queue()
qTrial.put("cheating+wife")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(5)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkcombo.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    print(soup.prettify())
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "grid-apps mb-10"})
    

In [ ]:
# FOR APKPURE.COM
qTrial = Queue()
qTrial.put("Starting hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    checkMore = 1
    firstCheck = 1
    interval = 0
    appIDList = ""
    first = 0
    while(checkMore):
        if(firstCheck == 0):
            interval = interval + 15
        payload = {'q': word,  't': 'app', 'begin': interval};
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        r = requests.get('https://apkpure.com/search-page', params=payload_str);
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup.prettify())
        names_table = soup.find_all("dl", attrs={"class": "search-dl"})
        #print(names_table)
        if(len(names_table) == 0):
            print("Skipping " + word)
            break
        for name in names_table:
            dtPart = name.find_all("dt");
            ddPart = name.find_all("dd");
            aPart = dtPart[0].find_all("a");
            appID = aPart[0]['href']
            imagePart = aPart[0].find_all("img");
            imageLink = imagePart[0]['src']
            titlePart = ddPart[0].find_all("p", attrs={"class": "search-title"})
            title = titlePart[0].find("a").get_text();
            starsPart = ddPart[0].find_all("span", attrs={"class": "score"})
            stars = starsPart[0]['title']
            stars = stars.rsplit(' ', 1)[1]
            pParts = ddPart[0].find_all("p")
            developerPart = pParts[1].find_all("a")
            developerName = developerPart[0].get_text()
            if first != 0:
                appIDList = appIDList + ","
            appIDList = appIDList + appID
            first = 1
            
            appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apkpure.com');
            insertIntoAppDetails(conn, appTableEntry)
        print("Done")
        firstCheck = 0
        if(len(names_table) == 0):
            checkMore = 0
    appMainTableEntry = (word, appIDList, 'null', 'apkpure.com');
#     print(appMainTableEntry)
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break

In [ ]:
# FOR APK-DL.COM

qTrial = Queue()
qTrial.put("cheating+wife")
qTrial.put("hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
    r = requests.get('https://apk-dl.com/search', params=payload_str);
    soup = BeautifulSoup(r.text, 'html.parser')
#     print(soup.prettify())
    names_table = soup.find_all("div", attrs={"class": "card no-rationale square-cover apps small"})
    appIDList = ""
    first = 0
    for name in names_table:
        appIDPart = name.find("a", attrs={"class": "card-click-target"})
        appID = appIDPart['href']
#         print(appID)
        imageLinkPart = name.find("img", attrs={"class": "cover-image lazy"})
        imageLink = imageLinkPart['data-original']
#         print(imageLink)
        titlePart = name.find("a", attrs={"class": "title"})
        title = titlePart.get_text()
#         print(title)
        developerNamePart = name.find("a", attrs={"class": "subtitle"})
        developerName = developerNamePart.get_text()
#         print(developerName)
        starsPart = name.find("div", attrs={"class" : "current-rating"})
        stars = starsPart['style']
        stars = stars.rsplit(' ', 1)[1]
        stars = stars[:-2]
        stars = int(stars)/10
#         print(stars)
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1

        appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apk-dl.com')
        insertIntoAppDetails(conn, appTableEntry)
    
    appMainTableEntry = (word, appIDList, 'null', 'apk-dl.com')
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    

In [11]:
# For APKMONK.com

qTrial = Queue()
qTrial.put("cheating+wife")
qTrial.put("hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
    r = requests.get('https://www.apkmonk.com/search', params=payload_str);
    soup = BeautifulSoup(r.text, 'html.parser')
    
#     print(soup.prettify())
    names_table = soup.find_all("div", attrs={"class": "col l6 m12 s12 xl4 faceout-row"})
    appIDList = ""
    first = 0
    for name in names_table:
        print(name)
    break

Starting cheating+wife 0 with queue length 1557
